In [1]:
!pip install transformers peft bitsandbytes trl deepeval optimum auto-gptq langchain torch flask
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 32.9 MB/s eta 0:00:00
     ━━

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig
from trl import SFTTrainer

#Quantize 4bits

In [4]:
compute_dtypes = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtypes,
)

In [2]:
# from google.colab import userdata
# userdata.get('Llama-3-8B')

In [6]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [7]:
# Ensure local directory does not exist
!rm -rf meta-llama/Meta-Llama-3-8B

model_id = "meta-llama/Meta-Llama-3-8B"
access_token = "#"

# Pass the access token to the tokenizer as well
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_id, token = access_token, quantization_config = quant_config,  device_map={"": 0})

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=500)
hf = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [8]:
prompt_template = PromptTemplate(
    input_variables=["product_name", "product_description", "interest_level", "user_name", "tone"],
    template="""
Write a personalized marketing email in a paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices, discount codes, or dates:

Product Name: {product_name}
Product Description: {product_description}
Interest Level: {interest_level}
User Name: {user_name}
Email Tone: {tone}

Email Content in Paragraph:
Subject: Exclusive {product_name} Offer Just for You!

Dear {user_name},
"""
)

email_chain = LLMChain(llm=hf, prompt=prompt_template)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
import re
# Define the function to generate the email
def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"
    validity_date = "1 DECEMBER 2024"

    # Determine discount offer and coupon code based on interest level
    discount_map = {
        "very interested": ("30%", "NEPAL30"),
        "interested": ("20%", "NEPAL20"),
        "somewhat interested": ("15%", "NEPAL15"),
        "not interested": ("10%", "NEPAL10")
    }
    discount_offers, coupon_code = discount_map.get(interest_level, ("10%", "NEPAL10"))  # Default

    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email in paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices of the product, discount codes, or dates:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Interest Level: {interest_level}\n"
        f"User Name: {user_name}\n"
        f"Email Tone: {tone}\n"
        f"Email Content in Paragraph:\nSubject: Exclusive {product_name} Offer Just for You!\n\n\n"
        f"Dear {user_name},\n\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    while True:  # Loop for validation
        # Generate the output
        outputs = model.generate(
            inputs.input_ids.to('cuda'),
            max_length=500,
            min_length=200,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
        )

        # Decode and process the email
        email = tokenizer.decode(outputs[0], skip_special_tokens=True)
        email_content_start = email.find("Subject:")
        generated_email = email[email_content_start:].strip()


        # Remove any existing sign-offs
        sign_offs = ["Regards,", "Best regards,", "Sincerely,", "Yours truly,", "Best", "money would be refunded", "company would be liable", "product can be returned after used", "simply use the discount"]
        for sign_off in sign_offs:
            if sign_off in generated_email:
                generated_email = generated_email.split(sign_off)[0].strip()

        # Guard against generated coupon codes
        forbidden_phrases = ["code", "coupon code", "discount code", "offer code", "Discount Offer", "retailed price", "retailed rate","discounted price", "discounted rate", "price", "product price", " 30-day money-back guarantee"]
        for phrase in forbidden_phrases:
            if phrase in generated_email:
                generated_email = generated_email.split(phrase)[0].strip()


        # Filter and remove unwanted elements
        generated_email = re.sub(r'http\S+', '', generated_email)
        generated_email = re.sub(r'www\.\S+', '', generated_email)
        generated_email = re.sub(r'<.*?>', '', generated_email)
        generated_email = re.sub(r'\d+% off', '', generated_email)

        # Append template values with dynamic coupon code and discount offer
        generated_email += (
            f"\n\nUse the coupon code '{coupon_code}' to get {discount_offers} off. Valid till {validity_date}.\n"
            f"\nBest regards,\n"
            f"{sender_name}\n"
            f"{company_name}\n"
            f"Visit us: {website_url}\n"
        )

        # Validation check
        if is_valid_email(generated_email):
            return generated_email
        else:
            print("Email not valid. Regenerating...")



In [10]:
def is_valid_email(email):
    # Customization based on my requirement
    forbidden_phrases = ["money would be refunded", "company would be liable",
                         "product can be returned after used", "simply use the discount", "sales", "promotions", "discounts", "prices", "price" ,  "discount code", "discount" , "dates", "Expiration Date" , "regular price", "Sincerely", "The SuperWidgets Team", "Yours sincerely", "Name of Company"]
    if any(phrase in email for phrase in forbidden_phrases):
        return False
    if len(email.split()) < 100 or len(email.split()) > 200:
        return False
    return True



In [11]:
# example for the generation
product_name = "SuperWidget"
product_description = "The SuperWidget is an innovative tool that helps you increase productivity and efficiency."
interest_level = "very interested"
user_name = "John Doe"

personalized_email = generate_email_auto(product_name, product_description, interest_level, user_name)
print(personalized_email)

Email not valid. Regenerating...
Subject: Exclusive SuperWidget Offer Just for You!


Dear John Doe,


You have been selected to receive an exclusive offer on the SuperWidget, which is an incredible tool that can help you increase your productivity and save time. The Super Widget is a unique tool that is designed to help you achieve your goals and objectives. It is a tool that has been designed to make your life easier and more efficient.


The Super Widget comes with a

Use the coupon code 'NEPAL30' to get 30% off. Valid till 1 DECEMBER 2024.

Best regards,
Binit Sapkota
MrBeerGod Technologies
Visit us: https://barahsinghe.com/



In [12]:
# Define the function to generate the email
def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"
    validity_date = "1 DECEMBER 2024"

    # Determine discount offer and coupon code based on interest level
    discount_map = {
        "very interested": ("30%", "NEPAL30"),
        "interested": ("20%", "NEPAL20"),
        "somewhat interested": ("15%", "NEPAL15"),
        "not interested": ("10%", "NEPAL10")
    }
    discount_offers, coupon_code = discount_map.get(interest_level, ("10%", "NEPAL10"))
    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")

    # Prepare the input for the LLMChain
    input_data = {
        "product_name": product_name,
        "product_description": product_description,
        "interest_level": interest_level,
        "user_name": user_name,
        "tone": tone
    }
    # Extract the generated text from the dictionary
    generation_result = email_chain.run(input_data)
    # Post-process the generated email content
    email_content = generation_result.strip()
    # Append personalized closing
    personalized_email = f"""
{email_content}

Use the coupon code '{coupon_code}' to get {discount_offers} off. Valid till {validity_date}.

Best regards,
{sender_name}
{company_name}
Visit us: {website_url}
"""
    return personalized_email

# Example usage for the email generation
product_name = "SuperWidget"
product_description = "The SuperWidget is an innovative tool that helps you increase productivity and efficiency."
interest_level = "very interested"
user_name = "John Doe"

personalized_email = generate_email_auto(product_name, product_description, interest_level, user_name)
print(personalized_email)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Write a personalized marketing email in a paragraph form with the following details, but do not generate information about sales, promotions, discounts, prices, discount codes, or dates:

Product Name: SuperWidget
Product Description: The SuperWidget is an innovative tool that helps you increase productivity and efficiency.
Interest Level: very interested
User Name: John Doe
Email Tone: excited and convincing

Email Content in Paragraph:
Subject: Exclusive SuperWidget Offer Just for You!

Dear John Doe,
We are excited to offer you a special discount on the SuperWidget, an innovative tool that will help you increase productivity and efficiency.

The SuperWidget is a must-have tool for any professional looking to stay ahead of the game. It is designed to help you manage your time more efficiently, increase your productivity, and stay organized.

We are offering a special discount for you, as a valued customer. The SuperWidget is normally priced at $199, but we are offering it to you for

In [18]:

# Define the function to generate the email
def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"
    validity_date = "1 DECEMBER 2024"

    # Determine discount offer and coupon code based on interest level
    discount_map = {
        "very interested": ("30%", "NEPAL30"),
        "interested": ("20%", "NEPAL20"),
        "somewhat interested": ("15%", "NEPAL15"),
        "not interested": ("10%", "NEPAL10")
    }
    discount_offers, coupon_code = discount_map.get(interest_level, ("10%", "NEPAL10"))
    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")

    # Prepare the input for the LLMChain
    input_data = {
        "product_name": product_name,
        "product_description": product_description,
        "interest_level": interest_level,
        "user_name": user_name,
        "tone": tone
    }

    # Try generating the email multiple times if needed
    max_attempts = 10
    for attempt in range(max_attempts):
        # Generate the email content
        generation_result = email_chain.run(input_data)
        email_content = generation_result.strip()

        # Apply restrictions to the generated email content
        email_content = apply_restrictions(email_content)

        # Append personalized closing and validate the email
        personalized_email = append_template(email_content, coupon_code, discount_offers, sender_name, company_name, website_url, validity_date)

        if is_valid_email(personalized_email):
            return personalized_email
        else:
            print(f"Email not valid on attempt {attempt + 1}. Regenerating...")

    return "Failed to generate a valid email after multiple attempts."


In [19]:

def apply_restrictions(generated_email):
    # Remove specific sign-offs
    sign_offs = ["Regards,", "Best regards,", "Sincerely,", "Yours truly,", "Best"]
    for sign_off in sign_offs:
        generated_email = generated_email.split(sign_off)[0].strip()

    # Guard against forbidden phrases and remove them
    forbidden_phrases = ["code", "coupon code", "discount code", "offer code", "Discount Offer",
                         "retailed price", "retailed rate", "discounted price", "discounted rate",
                         "price", "product price", "30-day money-back guarantee",
                         "money would be refunded", "company would be liable",
                         "product can be returned after used", "simply use the discount",
                         "sales", "promotions", "discounts", "prices", "price",
                         "discount code", "discount", "dates", "Expiration Date",
                         "regular price", "Sincerely", "The SuperWidgets Team",
                         "Yours sincerely", "Name of Company"]
    for phrase in forbidden_phrases:
        if phrase in generated_email:
            print(f"Removing forbidden phrase: {phrase}")
            generated_email = generated_email.split(phrase)[0].strip()

    # Clean up email content (remove links, HTML tags, etc.)
    generated_email = re.sub(r'http\S+|www\.\S+|<.*?>|\d+% off', '', generated_email)

    return generated_email


In [20]:

def append_template(generated_email, coupon_code, discount_offers, sender_name, company_name, website_url, validity_date):
    # Append the template values with dynamic coupon code and discount offer
    generated_email += (
        f"\n\nUse the coupon code '{coupon_code}' to get {discount_offers} off. Valid till {validity_date}.\n"
        f"\nBest regards,\n"
        f"{sender_name}\n"
        f"{company_name}\n"
        f"Visit us: {website_url}\n"
    )

    return generated_email



In [21]:

def is_valid_email(email):
    # Customization based on requirement
    forbidden_phrases = ["money would be refunded", "company would be liable",
                         "product can be returned after used", "simply use the discount", "sales", "promotions", "discounts", "prices", "price",
                         "discount code", "discount", "dates", "Expiration Date",
                         "regular price", "Sincerely", "The SuperWidgets Team",
                         "Yours sincerely", "Name of Company"]
    for phrase in forbidden_phrases:
        if phrase in email:
            print(f"Invalid phrase found in email: {phrase}")
            return False
    if len(email.split()) < 100 or len(email.split()) > 200:
        print("Email length is not within the valid range.")
        return False
    return True


In [22]:

product_name = "SuperWidget"
product_description = "The SuperWidget is an innovative tool that helps you increase productivity and efficiency."
interest_level = "very interested"
user_name = "John Doe"

personalized_email = generate_email_auto(product_name, product_description, interest_level, user_name)
print(personalized_email)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Removing forbidden phrase: code
Removing forbidden phrase: price
Removing forbidden phrase: sales
Email length is not within the valid range.
Email not valid on attempt 1. Regenerating...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Removing forbidden phrase: code
Removing forbidden phrase: price
Removing forbidden phrase: sales
Email length is not within the valid range.
Email not valid on attempt 2. Regenerating...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Removing forbidden phrase: code
Removing forbidden phrase: price
Removing forbidden phrase: sales
Email length is not within the valid range.
Email not valid on attempt 7. Regenerating...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Removing forbidden phrase: code
Removing forbidden phrase: price
Removing forbidden phrase: sales
Email length is not within the valid range.
Email not valid on attempt 8. Regenerating...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Removing forbidden phrase: code
Removing forbidden phrase: price
Removing forbidden phrase: sales
Email length is not within the valid range.
Email not valid on attempt 9. Regenerating...
Removing forbidden phrase: code
Removing forbidden phrase: price
Removing forbidden phrase: sales
Email length is not within the valid range.
Email not valid on attempt 10. Regenerating...
Failed to generate a valid email after multiple attempts.
